# This notebook contains some examples for how to use the finetune API.

In [2]:
"""In this example, we load print the Hugging Face URL for the best-model in the SN9_MODEL competition."""

import asyncio

import finetune as ft
from competitions.data import CompetitionId

# Each model competes in a single competition. Find the best top performing miner UID for the 
# competition we care about (SN9_MODEL, in this example).
top_model_uid = ft.graph.best_uid(competition_id=CompetitionId.SN9_MODEL)

# Get the HuggingFace URL for this model.
repo_url = asyncio.run(ft.mining.get_repo(top_model_uid))
print(f"The best-performing model for SN9_MODEL competition is {repo_url}")

2024-07-21 20:45:16.231 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-07-21 20:45:16.233 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-07-21 20:45:16.233 |     WARNING      | In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it.
2024-07-21 20:45:17.222 |       INFO       | Connected to finney network and wss://entrypoint-finney.opentensor.ai:443.
2024-07-21 20:45:25.817 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-07-21 20:45:25.818 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-07-21 20:45:25.818 |     WARNING      | I

In [3]:
"""In this example, we load the top model for the SN9_MODEL competition and converse with it."""

import asyncio

import bittensor as bt
import torch
from taoverse.model.competition import utils as competition_utils
from transformers import GenerationConfig

import constants
import finetune as ft
from competitions.data import CompetitionId

# The device to run the model on.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Download the top model to the specified directory.
download_dir = "./finetune-example"
model = asyncio.run(
    ft.mining.load_best_model(
        download_dir=download_dir, competition_id=CompetitionId.SN9_MODEL
    )
)

# Load the competition so we can load the right tokenizer.
metagraph = bt.metagraph(constants.SUBNET_UID)
competition = competition_utils.get_competition_for_block(CompetitionId.SN9_MODEL, metagraph.block, constants.COMPETITION_SCHEDULE_BY_BLOCK)
tokenizer = ft.model.load_tokenizer(competition.constraints)

# Decide on a prompt.
prompt = "How much wood could a woodchuck chuck if a woodchuck could chuck wood?"

# Tokenize it.
conversation = [{"role": "user", "content": prompt}]
input_ids = tokenizer.apply_chat_template(
    conversation,
    truncation=True,
    return_tensors="pt",
    max_length=competition.constraints.sequence_length,
    add_generation_prompt=True,
)

# Generate the output.
# You may wish to customize the generation config.
generation_config = GenerationConfig(
    max_length=competition.constraints.sequence_length,
    do_sample=True,
    temperature=0.8,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)
with torch.inference_mode():
    model.to(device)
    model.eval()
    input_ids = input_ids.to(device)
    output = model.generate(
        input_ids=input_ids, generation_config=generation_config
    )
    response = tokenizer.decode(
        output[0][len(input_ids[0]) :], skip_special_tokens=True
    )
    
    print(response)

2024-07-21 20:48:23.609 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-07-21 20:48:23.610 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-07-21 20:48:23.610 |     WARNING      | In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it.
2024-07-21 20:48:24.602 |       INFO       | Connected to finney network and wss://entrypoint-finney.opentensor.ai:443.
2024-07-21 20:48:33.468 |       INFO       | You are connecting to finney network with endpoint wss://entrypoint-finney.opentensor.ai:443.
2024-07-21 20:48:33.469 |     WARNING      | We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network.
2024-07-21 20:48:33.469 |     WARNING      | I